# Calculate Age (.gs)

Reads from Age sheet and updates the age column

In [ ]:
function Fill_age() {
  var ss = SpreadsheetApp.getActiveSpreadsheet();
  var sheet = ss.getActiveSheet();
  var ageSheet = ss.getSheetByName("Age");
  
  var dataRange = sheet.getRange(2, 1, sheet.getLastRow() - 1, sheet.getLastColumn());
  var data = dataRange.getValues();
  var ageData = ageSheet.getDataRange().getValues();
  
  var dobDict = {}; 
  var notFoundIds = []; 
  
  for (var i = 0; i < ageData.length; i++) {
    dobDict[ageData[i][0]] = new Date(ageData[i][1]);
  }

  for (var i = 0; i < data.length; i++) {
    var participantId = data[i][0];
    var ageCell = data[i][4];
    if ((ageCell === "" || ageCell === "DOB not found") && participantId) { 
      var dob = dobDict[participantId];
      if (dob) {
        var currentDate = new Date(data[i][1]);
        var age = getAge(dob, currentDate); 
        data[i][4] = age; 
      } else {
        data[i][4] = "DOB not found"; 
        notFoundIds.push(participantId); 
      }
    }
  }
  
 
  var ageColumnIndex = 5; 
  var ageValues = data.map(function(row) {
    return [row[ageColumnIndex - 1]]; 
  });
  
  sheet.getRange(2, ageColumnIndex, ageValues.length).setValues(ageValues);
  
  if (notFoundIds.length > 0) {
    var message = "DOB not found for ID(s): " + notFoundIds.join(", ");
    SpreadsheetApp.getUi().alert(message);
  } else {
    SpreadsheetApp.getUi().alert('Age filled for all applicable entries.');
  }

  function getAge(dob, currentDate) {
    var age = currentDate.getFullYear() - dob.getFullYear();
    var m = currentDate.getMonth() - dob.getMonth();
    if (m < 0 || (m === 0 && currentDate.getDate() < dob.getDate())) {
      age--;
    }
    return age;
  }
}
